In [1]:
from eda import get_objectives, get_constraints, non_dominated_sort, assign_crowding_distance, binary_tournament_selection, sample_population, cleanupsamples, generate_example_data, organize_results

In [2]:
from numpy import random
import numpy as np
from scipy.stats import gamma, norm
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.stats import multivariate_normal as mvn
from scipy.spatial.distance import jensenshannon

In [3]:
def transform_items_to_z(items):
    alpha = np.empty(items.shape[1])
    beta = np.empty(items.shape[1])
    items_z = np.empty(items.shape)
    for i in range(items.shape[1]):
        a, loc, scale = gamma.fit(items[:, i], floc=0.0)
        alpha[i] = a
        beta[i] = scale
        u = gamma.cdf(items[:,i], a = a, scale = scale)
        u = np.clip(u, 1e-12, 1-1e-12)
        items_z[:,i] = norm.ppf(u) 
    return alpha, beta, items_z

In [4]:
def convert_to_long(YY, n_selected, n_obj, n_con):
    rows = []
    for j in range(n_selected):
        for k in range(n_obj+n_con):
            if k < n_obj:
                rows.append(pd.DataFrame({
                    'cumulative': YY[:,j, k],
                    'step': f'Step {j}',
                    'objective': f'Obj {k}'
                }))
            else:
                rows.append(pd.DataFrame({
                    'cumulative': YY[:,j, k],
                    'step': f'Step {j}',
                    'objective': f'Con {k-n_obj}',
                }))
    df_Y = pd.concat(rows, ignore_index=True)
    return df_Y

def ecdf(df_table,n_selected,n_obj,n_con):
    nk = len(df_table)/(n_selected*(n_obj+n_con))
    ecdf_table = np.zeros((n_selected,(n_obj+n_con),int(nk)))
    for t in range(1,n_selected):
        for i in range(n_obj+n_con):
            if i < n_obj:
                ecdf_table[t,i,:] = np.sort(df_table['cumulative'][(df_table['objective'] == f'Obj {i}')&(df_table['step'] == f'Step {t}')].values)
            else:
                ecdf_table[t,i,:] = np.sort(df_table['cumulative'][(df_table['objective'] == f'Con {i - n_obj}')&(df_table['step'] == f'Step {t}')].values)
    return ecdf_table

def z_transform_from_ecdf(Y, ecdf_table):
    N, d = Y.shape  
    Y_z = np.empty_like(Y)
    for t in range(1,N):
        for i in range(d):
            ranks = np.searchsorted(ecdf_table[t,i,:], Y[t, i], side='right')
            u = ranks / len(ecdf_table[t,i,:])
            u = np.clip(u, 1e-12, 1 - 1e-12)
            Y_z[t, i] = norm.ppf(u)
    return Y_z

In [5]:
def get_norm_cumu_objectives(items, items_z, population, n_selected, n_obj, n_con, rng, if_inital): 
# can modify to use objectives instead of population
    XX = np.empty((population.shape[0], n_selected, n_obj+n_con))
    XX_z = np.empty((population.shape[0], n_selected, n_obj+n_con))
    for k in range(population.shape[0]):
        if if_inital:
            qx = rng.permutation(population[k, :]) # permutation?
        else:
            qx = population[k, :]
        XX[k,:,:] = items[qx,:]
        XX_z[k,:,:] = items_z[qx,:]
    YY = np.cumsum(XX,axis = 1)
    df_Y = convert_to_long(YY, n_selected, n_obj, n_con)
    ecdf_table = ecdf(df_Y, n_selected, n_obj, n_con)
    YY_z = np.empty_like(YY)
    for k in range(YY.shape[0]):
        YY_z[k] = z_transform_from_ecdf(YY[k], ecdf_table)
        YY_z[k,0,:] = XX_z[k,0,:]
    return XX, XX_z, YY, YY_z

In [6]:
def fit_markov_in_y_by_t(X, Y):
    K, N, d = X.shape
    A_list = np.zeros((N-1, d, d))
    b_list = np.zeros((N-1, d))
    Q_list = np.zeros((N-1, d, d))
    R2_list = np.zeros(N-1)
    reg_list = []

    for t in range(1, N):  
        S_t = Y[:, t-1, :]  
        Z_t = X[:, t,   :] 
        reg_t = LinearRegression(fit_intercept=True)
        reg_t.fit(S_t, Z_t)
        A_t = reg_t.coef_      # (d, d)
        b_t = reg_t.intercept_ # (d,)
        Z_hat_t = reg_t.predict(S_t)
        R_t = Z_t - Z_hat_t
        Q_t = np.cov(R_t, rowvar=False, bias=False)
        r2 = reg_t.score(S_t, Z_t)

        A_list[t-1, :, :] = A_t
        b_list[t-1, :] = b_t
        Q_list[t-1, :, :] = Q_t
        R2_list[t-1] = r2
        reg_list.append(reg_t)
    params = {"A": A_list,"b": b_list,"Q": Q_list,"regs": reg_list,"R2": R2_list}
    return params

In [7]:
def fit_conditional(items, population, n_selected, n_obj, n_con, rng, if_inital):
    _, _, z_items = transform_items_to_z(items)
    objectives, objectives_z, cumu_objectives, cumu_objectives_z = get_norm_cumu_objectives(items, z_items, population, 
                                                                                                n_selected, n_obj, n_con, 
                                                                                                rng, if_inital)
    dist_params = fit_markov_in_y_by_t(objectives_z, cumu_objectives_z)
    return z_items, objectives_z, dist_params

In [ ]:
def conditional_density_given_Y_and_t(X_candidates, y_normal, params_time, t):
    A_all = params_time["A"]  
    b_all = params_time["b"]  
    Q_all = params_time["Q"]  

    A_t = A_all[t-1]
    b_t = b_all[t-1]
    Q_t = Q_all[t-1]
    X_candidates = np.asarray(X_candidates)
    y_normal = np.asarray(y_normal).reshape(-1)
    mean_t = A_t @ y_normal + b_t

    densities = mvn.pdf(X_candidates, mean=mean_t, cov=Q_t)
    return densities

In [9]:
def base_rate_model(items_z, XX_0):
    mean0 = np.mean(XX_0, axis = 0)
    Sigma0 = np.cov(XX_0.T) 
    mvn0 = mvn(mean=mean0, cov=Sigma0)
    x_candidates = items_z
    probabilities = mvn0.pdf(x_candidates)
    probabilities = (probabilities+1e-12)/sum(probabilities+1e-12)
    return probabilities

In [18]:
def sample_population_conditional(
    samples, samples_z, objectives_z, dist_params, 
    pop_size, n_selected, capacity, rng): # no use of rng

    pop_count = 0
    population = np.zeros((pop_size, n_selected), dtype=np.int32)
    n_items = samples.shape[0]

    while pop_count < pop_size:
        # select sequentially
        knapsack = np.zeros(n_selected, dtype=int) # here knapsack is knapsack indices
        for n in range(n_selected):
            if n == 0: # select first item
                probabilities = base_rate_model(samples_z, objectives_z[:, 0, :])
                first_choice = rng.choice(n_items, p=probabilities)
                first_item = samples_z[first_choice,:]
                x_indices = np.setdiff1d(np.arange(n_items), knapsack)
                y_prev = first_item 
                knapsack[0] = first_choice
            else:
                x_indices = np.setdiff1d(np.arange(n_items), knapsack[:n])
                x_candidates = samples_z[x_indices, :]
                densities = conditional_density_given_Y_and_t(
                    x_candidates, y_prev, dist_params, n
                )
                probabilities = densities/sum(densities)
                next_choice = rng.choice(len(probabilities), p=probabilities)
                next_index = x_indices[next_choice]
                next_item = samples_z[next_index,:]
                knapsack[n] = next_index
                y_prev = y_prev + next_item
        
        constraint = np.sum(samples[knapsack, -1])
        if constraint <= capacity:
            population[pop_count, :] = knapsack
            pop_count += 1
    
    return population

In [16]:
class KnapsackEDA:
    def __init__(self, items, capacity, n_selected, n_obj, n_con, pop_size=1000, 
                 generations=5, max_no_improve_gen=20, seed=1123):
        self.items = items
        self.items_z = None
        self.capacity = capacity
        self.n_selected = n_selected
        self.n_obj = n_obj
        self.n_con = n_con
        self.pop_size = pop_size
        self.generations = generations
        self.max_no_improve_gen = max_no_improve_gen
        self.rng = random.default_rng(seed=seed)
        self.if_inital = True
        
        # self.distribution = None
        self.distribution_params = None
        self.selected_population = None  # (pop_size, n_selected)
        self.selected_objectives = None  # (pop_size, n_obj) objective values are summed over solutions
        self.objectives_z = None  # (pop_size, n_selected, n_obj)
        self.distribution_params_table = []
        self.pareto_indices_table = []
        self.pareto_front_table = []
    
    def _generate_initial_population(self):
        n_items = self.items.shape[0]
        distribution = np.ones(n_items) / n_items
        population = sample_population(
            self.items, distribution, self.pop_size, self.n_selected, 
            self.capacity, self.rng
        )
        objectives = get_objectives(self.items, population, self.n_obj)
        
        ranks, fronts = non_dominated_sort(objectives)
        distances_all_solutions = np.zeros(population.shape[0], dtype=float)
        for f in fronts:
            distances = assign_crowding_distance(objectives[f, :])
            distances_all_solutions[f] = distances
        
        select_indices = np.array([], dtype=int)
        while len(select_indices) < self.pop_size:
            indice = binary_tournament_selection(
                population, ranks, distances_all_solutions, self.rng
            )
            select_indices = np.concatenate([select_indices, np.array([indice])])
        
        selected_population = population[select_indices]
        selected_objectives = objectives[select_indices]

        items_z, objectives_z, distribution_params = fit_conditional(self.items, selected_population, 
                                                                        self.n_selected, self.n_obj, self.n_con,
                                                                        self.rng, self.if_inital) # may need a different rng
        
        return selected_population, selected_objectives, items_z, objectives_z, distribution_params
    
    def _update_distribution(self):
        population = sample_population_conditional(
            self.items, self.items_z, self.objectives_z, self.distribution_params, 
            self.pop_size, self.n_selected, self.capacity, self.rng
        )
        objectives = get_objectives(self.items, population, self.n_obj)
        
        # Find current pareto front
        # _, _, _, fronts_current = non_dominated_sort(objectives)
        _, fronts_current = non_dominated_sort(objectives)
        pareto_indices = population[fronts_current[0]]
        
        objectives = np.vstack((self.selected_objectives, objectives))
        population = np.vstack((self.selected_population, population))
        
        # _, _, ranks, fronts = non_dominated_sort(objectives)
        ranks, fronts = non_dominated_sort(objectives)
        select_indices = np.array([], dtype=np.int32)
        for f in fronts:
            if len(select_indices) + len(f) <= self.pop_size:
                select_indices = np.concatenate([select_indices, f])
            else:
                remaining_size = self.pop_size - len(select_indices)
                f_distance = assign_crowding_distance(objectives[f, :])
                sort_indices = np.argsort(f_distance)[::-1]
                remaining = f[sort_indices[:remaining_size]]
                select_indices = np.concatenate([select_indices, remaining])
                break
        
        selected_population = population[select_indices]
        selected_objectives = objectives[select_indices]
        
        # update distribution
        items_z, objectives_z, distribution_params = fit_conditional(self.items, selected_population, 
                                                                        self.n_selected, self.n_obj, self.n_con,
                                                                        self.rng, self.if_inital)
        
        return distribution_params, selected_population, selected_objectives, pareto_indices

    def run(self):
        self.selected_population, self.selected_objectives, self.items_z, self.objectives_z, self.distribution_params = \
            self._generate_initial_population()
        self.if_inital = False
        
        # Run generations (fixed number of generations)
        for g in range(self.generations):
            print(f"Generation {g+1}/{self.generations}")
            self.distribution_params, self.selected_population, self.selected_objectives, \
                pareto_indices = self._update_distribution()
            
            pareto_front = np.zeros((pareto_indices.shape[0], self.items.shape[1]))
            for k in range(pareto_indices.shape[0]):
                pareto_front[k, :] = np.sum(self.items[pareto_indices[k, :], :], axis=0)
                
            self.distribution_params_table.append(self.distribution_params.copy())
            self.pareto_indices_table.append(pareto_indices.copy())
            self.pareto_front_table.append(pareto_front.copy())

        return {
            'distribution_params_table': self.distribution_params_table,
            'pareto_indices_table': self.pareto_indices_table,
            'pareto_front_table': self.pareto_front_table,
        }

    

In [12]:
#parameters
n_items = 60
n_selected = 6
n_obj = 3
n_con = 1
shape = [3.0, 4.0, 5.0, 8.0]
scale = [2.0, 3.0, 1.0, 1.0]
r = np.array([      
    [1.0, -0.3, -0.3, 0.6],
    [-0.3, 1.0, -0.3, 0.6],
    [-0.3, -0.3, 1.0, 0.6],
    [0.6, 0.6, 0.6, 1.0],
])
capacity = int(shape[-1]*scale[-1]*n_selected)
pop_size = 1000
generations = 5 # do not matter if check convergence
max_no_improve_gen = 10

In [13]:
items_seed = 1211
eda_seed = 1223
# Generate data
items, rpos = generate_example_data(r, shape, scale, n_items=n_items, seed=items_seed)

In [ ]:
# Run EDA
eda = KnapsackEDA(
    items=items,
    capacity=capacity,
    n_selected=n_selected,
    n_obj=n_obj,
    n_con=n_con,
    pop_size=pop_size,
    generations=generations,
    max_no_improve_gen=max_no_improve_gen,
    seed=eda_seed
)

#organize results    
results = eda.run()  